In [1]:
import pandas as pd
import numpy as np
import datetime as dt


In [2]:
usersDataTrain = pd.read_csv(r"C:\Users\brian\Documents\GitHub\TP2DATOS\Eventos(+completo).csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (4,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
eventosData = usersDataTrain
eventosData = eventosData[['person','event']]
eventosData = pd.get_dummies(eventosData, columns=['event'])
eventosData = eventosData.groupby('person').agg({'event_ad campaign hit':'sum', 'event_brand listing':'sum',
       'event_checkout':'sum', 'event_conversion':'sum', 'event_generic listing':'sum',
       'event_lead':'sum', 'event_search engine hit':'sum', 'event_searched products':'sum',
       'event_staticpage':'sum', 'event_viewed product':'sum', 'event_visited site':'sum'})

In [4]:
eventosData.replace(0,np.nan,inplace = True)
eventosData[eventosData.columns].mean()

event_ad campaign hit       5.999624
event_brand listing        11.249259
event_checkout              1.989310
event_conversion            1.651759
event_generic listing       6.359976
event_lead                  1.697755
event_search engine hit     4.079828
event_searched products     9.976018
event_staticpage            2.762950
event_viewed product       33.614974
event_visited site          5.336253
dtype: float64

In [5]:
#Vamos a mapear si son mayores al promedio de la columna
eventosData['usuario_Checkeador'] = eventosData['event_checkout'].map(lambda x: 1 if (x >=  1.989310) else 0)
eventosData['usuario_susceptible_campaign'] = eventosData['event_ad campaign hit'].map(lambda x: 1 if (x >=  5.999624) else 0)
eventosData['usuario_convertidor'] = eventosData['event_conversion'].map(lambda x: 1 if (x >=  1.651759) else 0)
eventosData['usuario_buscador'] = eventosData['event_viewed product'].map(lambda x:1 if (x>= 33.614974) else 0)
eventosData['usuario_dubitativo']= ((eventosData['usuario_Checkeador'] + eventosData['usuario_buscador'] )== 2)
eventosData['usuario_poco_util'] =((eventosData['usuario_Checkeador'] + eventosData['usuario_buscador'] )== 0)
eventosData['usuario_recurrente'] =  eventosData['event_visited site'].map(lambda x: 1 if (x >= 5.336253) else 0)
eventosData['usuario_muy_activo']=((eventosData['usuario_Checkeador'] + eventosData['usuario_buscador']+eventosData['usuario_convertidor'] )== 3)
eventosData = eventosData[['usuario_Checkeador','usuario_susceptible_campaign','usuario_buscador','usuario_dubitativo','usuario_poco_util','usuario_muy_activo','usuario_convertidor']]
eventosData = eventosData.fillna(0)

In [6]:
usersDataTrain = pd.read_csv("setEntrenamiento.csv")
usersDataTrain = pd.merge(usersDataTrain,eventosData,how = "left",on = "person")
usersDataTrain=usersDataTrain.fillna(0)
usersDataTrain.to_csv("setEntrenamiento.csv",index = False)

In [7]:
test = pd.read_csv('setTest.csv')
test = pd.merge(test,eventosData,how = "left",on = "person")
test =test.fillna(0)
test.to_csv("setTest.csv",index = False)